# Chapter 19: Discrete Optimization

In [1]:
import math
import numpy as np
from dataclasses import dataclass
from multipledispatch import dispatch
from itertools import combinations

## Algorithm 19.1

In [2]:
@dataclass
class MixedIntegerProgram:
    A: np.ndarray
    b: np.ndarray
    c: np.ndarray
    D: list

## Algorithm 19.2

In [3]:
relax = lambda MIP: LinearProgram(MIP.A, MIP.b, MIP.c) # from algorithm 11.1
def round_ip(MIP):
    x = minimize_lp_init(relax(MIP)) # from algorithm 11.5
    for i in MIP.D:
        x[i] = int(x[i])
    return x

## Algorithm 19.3

In [4]:
isint = lambda x, epsilon=1e-10 : abs(round(x)-x) <= epsilon
@dispatch(np.ndarray)
def is_totally_unimodular(A):
    if np.any([a not in [0,-1,1] for a in A.reshape(-1,)]):
        return False
    r, c = A.shape
    for i in range(min([r,c])):
        for a in list(combinations(np.arange(0,r,1),i+1)):
            for b in list(combinations(np.arange(0,c),i+1)):
                B = A[np.ix_(list(a), list(b))]
                if np.linalg.det(B) not in [0,-1,1]:
                    return False
    return True

@dispatch(MixedIntegerProgram)
def is_totally_unimodular(MIP):
    return is_totally_unimodular(MIP.A) and np.all(isint(MIP.b)) and np.all(isint(MIP.c))

### Example

In [5]:
A_1 = np.array([
    [1, 0, 1],
    [0,0,0],
    [1, 0, -1]
])

print(f"1st Matrix is unimodular? -> {is_totally_unimodular(A_1)}")

A_2 = np.array([
    [1, 0, 1],
    [0,0,0],
    [1, 0, 0]
])

print(f"2nd Matrix is unimodular? -> {is_totally_unimodular(A_2)}")

A_3 = np.array([
    [-1, -1, 0, 0, 0],
    [1, 0, -1, -1, 0],
    [0, 1, 1, 0, -1]
])

print(f"3rd Matrix is unimodular? -> {is_totally_unimodular(A_3)}")


1st Matrix is unimodular? -> False
2nd Matrix is unimodular? -> True
3rd Matrix is unimodular? -> True


## Algorithm 19.4

In [6]:
frac = lambda x: math.modf(x)[0]
def cutting_plane(MIP):
    LP = relax(MIP)
    x, b_inds, v_inds = minimize_lp_init(LP) # from algorithm 11.5
    n_orig = len(x)
    D= MIP.D
    while not np.all([isint(x[i]) for i in D ]):
        AB, AV = LP.A[:, b_inds], LP.A[:, v_inds]
        Abar = np.linalg.solve(AB, AV)
        b = -1
        for i in D:
            if not isint(x[i]):
                b += 1
                A2 = np.concatenate((
                    np.concatenate((LP.A, np.zeros((LP.A.shape[0],1))),axis=1),
                    np.zeros((1, LP.A.shape[1]+1))
                ), axis=0)
                A2[-1, -1]= 1
                A2[-1, v_inds] = list(map(lambda x: np.floor(x) -x, Abar[b, :]))
                b2 = np.concatenate((LP.b, -np.array([frac(x[i])]).reshape(1,-1)),axis=0)
                c2 = np.concatenate((LP.c, np.array([[0]]) ), axis=0)
                LP = LinearProgram(A2,b2,c2)  # from algorithm 11.1
        x, b_inds, v_inds = minimize_lp_init(LP)
    return x[:n_orig]

### Example

#### Code from chapter 11

In [7]:
@dataclass
class LinearProgram:
    A : np.ndarray
    b: np.ndarray
    c: np.ndarray

def minimize_lp(B, LP):
    done = False
    while not done:
        B, done = step_lp(B, LP)
    return B

def step_lp(B, LP):
    A, b, c = LP.A, LP.b, LP.c
    n = A.shape[1]
    b_inds = np.sort(np.array(B)-1)
    n_inds = np.sort(np.setdiff1d(np.arange(0,n,1), np.array(B)-1))
    
    AB, AV = A[:, b_inds], A[:, n_inds]

    xB = np.linalg.solve(AB, b)
    cB = c[b_inds,:]
    lambda_ = np.linalg.solve(AB.T, cB)
    cV = c[n_inds]
    muV = cV - np.matmul(AV.T, lambda_)

    q, p, xq_prime, delta = -1, 0, np.infty, np.infty
    for i in range(len(muV)):
        if muV[i] < 0:
            pi, xi_prime = edge_transition(LP, B, i+1)
            if np.matmul(muV[i],xi_prime) < delta:
                q, p, xq_prime, delta = i+1, pi, xi_prime, np.matmul(muV[i],xi_prime)

    if q == -1:
        return B, True
    
    if xq_prime == np.infty:
        raise ValueError("unbounded")
    
    j = np.argmax(B == b_inds[p]+1)
    B[j] = n_inds[q-1] +1
    return B, False

def edge_transition(LP, B, q):
    A, b, c = LP.A, LP.b, LP.c
    n = A.shape[1]
    b_inds = np.sort(np.array(B)-1)
    n_inds = np.sort(np.setdiff1d(np.arange(0,n,1), np.array(B)-1))
    AB = A[:, b_inds]
    d, xB = np.linalg.solve(AB, A[:, n_inds[q-1]]), np.linalg.solve(AB, b)
 
    p, xq_prime = 0, np.infty

    for i in range(len(d)):
        if d[i] > 0:
            v = xB[i,:]/d[i]
            if v < xq_prime:
                p, xq_prime = i, v
    
    return p, xq_prime

#-#-#-# This function needed to be changed to be consistent with algo 19.4 #-#-#-#
def minimize_lp_init(LP):
    A, b, c = LP.A, LP.b, LP.c
    m, n = A.shape
    z = np.ones((m,1))
    Z = np.diag([1 if i >=0 else -1 for i in b])

    A_prime = np.concatenate((A,Z),axis=1)
    b_prime = b
    c_prime = np.concatenate((np.zeros((n,1)),z),axis=0)

    LP_init = LinearProgram(A_prime, b_prime, c_prime)
    B = np.arange(n+1, m+n+1,1)
    B = minimize_lp(B, LP_init)

    if np.any([True if i > n else False for i in B]):
        print("infeasible")

    return get_vertex(B, LP_init, n)

#-#-#-# This function needed to be changed to be consistent with algo 19.4 #-#-#-#
def get_vertex(B, LP, n):
    A, b, c = LP.A, LP.b, LP.c
    b_inds = np.sort(np.array(B)-1)
    AB = A[:, b_inds]
    xB = np.linalg.solve(AB, b)
    x = np.zeros(len(c))
    x[b_inds] = xB.reshape(-1,)

    return x[:n], [v for v in b_inds if v in range(n)], [v for v in range(n) if v not in b_inds]

#### Example 19.3

In [8]:
A = np.array([
    [0.5, -0.5, 1.0],
    [2.0, 0.5, -1.5]
])
b = np.array([[2.5], [-1.5]])
c = np.array([[2], [1], [3]])
D = [0, 1, 2]
MIP = MixedIntegerProgram (A, b, c, D)
cutting_plane(MIP)

array([1., 2., 3.])